# See if character lists are available for LitBank texts

In [2]:
# Build LitBank titles
import pandas as pd

fpath = '/data/storyq/litbank_titles.csv'
litbank_info = pd.read_csv(fpath, sep='\t+')
litbank_info

/usr0/home/mamille2/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """


,Gutenberg ID,Date,Author,Title
0,514,1868,"Alcott, Louisa May",Little Women
1,18581,1904,"Alger, Horatio, Jr.",Adrift in New York: Tom and Florence Braving t...
2,5348,1868,"Alger, Horatio, Jr.","Ragged Dick, Or, Street Life in New York with ..."
3,158,1815,"Austen, Jane",Emma
4,105,1818,"Austen, Jane",Persuasion
...,...,...,...,...
96,541,1920,"Wharton, Edith",The Age of Innocence
97,174,1890,"Wilde, Oscar",The Picture of Dorian Gray
98,2005,1919,"Wodehouse, P. G. (Pelham Grenville)",Piccadilly Jim
99,16357,1788,"Wollstonecraft, Mary",Mary: A Fiction
